In [1]:
import pandas as pd
from openai import OpenAI

from dotenv import find_dotenv,load_dotenv

_ = load_dotenv(find_dotenv())

In [2]:
data = pd.read_parquet('../data/actual/interim/data_for_db.parquet')

In [3]:
data.head(2)

,Вопрос из БЗ,Ответ из БЗ,Классификатор 1 уровня,Классификатор 2 уровня
0,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео
1,Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации


In [4]:
data = data.drop_duplicates(subset=['Вопрос из БЗ'])

In [5]:
questions = data['Вопрос из БЗ'].tolist() 

In [6]:
questions[:10]

['Что нельзя публиковать на RUTUBE?',
 'Почему могут отключить монетизацию из-за авторского права на видео и как этого избежать?',
 'Почему могут отключить монетизацию из-за искусственного завышения показателей на видео и как этого избежать? ',
 'Для каких статусов доступна монетизация, и почему могут отключить монетизацию из-за изменения юридического статуса на видео, как этого избежать?',
 'Какой контент можно использовать для монетизации, то есть в коммерческих целях?',
 'Какие материалы защищены авторским правом?',
 'Как подтвердить права на использование контента для монетизации?',
 'Кому принадлежат авторские права на контент, который я размещаю на платформе?',
 'Можно ли цитировать чужие произведения?',
 'Могу ли я использовать небольшие фрагменты фильмов для создания своих видео?']

In [8]:
prompts = [x + '\nВОПРОС: {question}' for x in ['Переформулируй вопрос, чтобы он другими словами спрашивал то же самое. Чтобы вопрос сохранил свой смысл, но звучал по-другому.',
'Переформулируй вопрос, как будто его задал подросток, который увлекается анимэ. Чтобы вопрос сохранил свой смысл, но звучал по-другому.',
'Переформулируй вопрос, как будто его задала пожилая женщина, которая увлекается садоводством. Чтобы вопрос сохранил свой смысл, но звучал по-другому.',
'Переформулируй вопрос, как будто его задал человек, ведущий здоровый образ жизни. Чтобы вопрос сохранил свой смысл, но звучал по-другому.',
'Переформулируй вопрос, как будто его задала фитоняшка. Чтобы вопрос сохранил свой смысл, но звучал по-другому.',
'Переформулируй вопрос, как будто его задал ребенок. Чтобы вопрос сохранил свой смысл, но звучал по-другому.',
'Переформулируй вопрос, как будто его задал гопник. Чтобы вопрос сохранил свой смысл, но звучал по-другому.',
'Переформулируй вопрос, как будто его задал начинающий программист. Чтобы вопрос сохранил свой смысл, но звучал по-другому.',
'Переформулируй вопрос, как будто его задал фанат рок-музыки. Чтобы вопрос сохранил свой смысл, но звучал по-другому.',
'Переформулируй вопрос, как будто его задал любитель шансона. Чтобы вопрос сохранил свой смысл, но звучал по-другому.',
'Переформулируй вопрос, как будто его задал геймер. Чтобы вопрос сохранил свой смысл, но звучал по-другому.',
'Переформулируй вопрос, как будто его задал многодетный отец-алиментщик. Чтобы вопрос сохранил свой смысл, но звучал по-другому.',
'Переформулируй вопрос, как будто его задала мамочка в декрете. Чтобы вопрос сохранил свой смысл, но звучал по-другому.']]


In [13]:
import json

def generate_jsonl(prompts, questions, output_file):
    with open(output_file, 'w', encoding='utf-8') as f:
        custom_id = 1
        for question in questions:
            for prompt in prompts:
                data = {
                    "custom_id": f"request-{custom_id}",
                    "method": "POST",
                    "url": "/v1/chat/completions",
                    "body": {
                        "model": "gpt-4o-mini",
                        "messages": [
                            {"role": "user", "content": prompt + f"\nВОПРОС: {question}"}
                        ],
                        "max_tokens": 1000
                    }
                }
                f.write(json.dumps(data, ensure_ascii=False) + '\n')
                custom_id += 1

# Генерация файла
generate_jsonl(prompts, questions, 'batchinput.jsonl')


In [15]:
from openai import OpenAI
client = OpenAI()

batch_input_file = client.files.create(
  file=open("batchinput.jsonl", "rb"),
  purpose="batch"
)


In [17]:
batch_input_file_id = batch_input_file.id

client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "nightly eval job"
    }
)

Batch(id='batch_66f74ad286fc819082a919bc30432410', completion_window='24h', created_at=1727482578, endpoint='/v1/chat/completions', input_file_id='file-o3k0QgCpT3arN11VKULc6O1e', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1727568978, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

Batch(id='batch_66f74ad286fc819082a919bc30432410', completion_window='24h', created_at=1727482578, endpoint='/v1/chat/completions', input_file_id='file-o3k0QgCpT3arN11VKULc6O1e', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1727568978, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


In [20]:
batch_input_file_id

'file-o3k0QgCpT3arN11VKULc6O1e'

In [22]:
from openai import OpenAI
client = OpenAI()

client.batches.retrieve("batch_66f74ad286fc819082a919bc30432410")

Batch(id='batch_66f74ad286fc819082a919bc30432410', completion_window='24h', created_at=1727482578, endpoint='/v1/chat/completions', input_file_id='file-o3k0QgCpT3arN11VKULc6O1e', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1727568978, failed_at=None, finalizing_at=None, in_progress_at=1727482581, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=3731))

In [30]:
from openai import AsyncOpenAI

client = AsyncOpenAI()

In [33]:
import openai
import asyncio
import pandas as pd
import os

# Установите ваш API-ключ
# openai.api_key = "YOUR_API_KEY"  # Замените на ваш API-ключ

# Асинхронная функция для отправки запроса к OpenAI API
async def complete(query: str, instruction: str):
    try:
        completion = await AsyncOpenAI().chat.completions.create(
            model="gpt-4o-mini",  # Замените на вашу модель, например gpt-4-turbo
            temperature=1,
            messages=[
                {"role": "system", "content": instruction},
                {"role": "user", "content": query}
            ]
        )
        return completion
    except Exception as e:
        print(f"Ошибка: {e}")
        return None

# Функция для извлечения содержимого ответа
def extract_content(completion) -> str:
    try:
        return completion.choices[0].message['content']
    except Exception:
        return '==ERROR=='

# Асинхронная функция обработки вопросов
async def process_questions(questions, prompts, output_file, n=10):
    new_questions = []

    # Загружаем данные, если файл уже существует
    if os.path.exists(output_file):
        existing_data = pd.read_parquet(output_file).values.tolist()
        new_questions.extend(existing_data)

    for idx, question in enumerate(questions):
        temp = []
        temp.append(question)

        tasks = []  # Список задач для каждого вопроса

        # Для каждого вопроса формируем асинхронные задачи для каждого промпта
        for prompt in prompts:
            task = complete(question, prompt)
            tasks.append(task)

        # Выполняем асинхронные задачи для одного вопроса
        responses = await asyncio.gather(*tasks)

        for response in responses:
            temp.append(extract_content(response))

        new_questions.append(temp)

        # Сохраняем каждые n итераций
        if (idx + 1) % n == 0:
            df = pd.DataFrame(new_questions)
            df.to_parquet(output_file)
            print(f"Сохранено {idx + 1} вопросов.")

    # Окончательное сохранение оставшихся данных
    df = pd.DataFrame(new_questions)
    df.to_parquet(output_file)
    print("Окончательное сохранение завершено.")

# Пример использования
if __name__ == "__main__":

    output_file = 'batch_output.parquet'

    # Запуск асинхронного процесса
    await process_questions(questions, prompts, output_file, n=10)


c:\Users\snake\projects\rutube_hackathon\.venv\Lib\site-packages\httpcore\_backends\anyio.py:115: RuntimeWarning: coroutine 'run_async_requests' was never awaited
  with anyio.fail_after(timeout):


Сохранено 10 вопросов.
Сохранено 20 вопросов.
Сохранено 30 вопросов.
Сохранено 40 вопросов.
Сохранено 50 вопросов.
Сохранено 60 вопросов.
Сохранено 70 вопросов.
Сохранено 80 вопросов.
Сохранено 90 вопросов.
Сохранено 100 вопросов.
Сохранено 110 вопросов.
Сохранено 120 вопросов.
Сохранено 130 вопросов.
Сохранено 140 вопросов.
Сохранено 150 вопросов.
Сохранено 160 вопросов.
Сохранено 170 вопросов.
Сохранено 180 вопросов.
Сохранено 190 вопросов.
Сохранено 200 вопросов.
Сохранено 210 вопросов.
Сохранено 220 вопросов.
Сохранено 230 вопросов.
Сохранено 240 вопросов.
Сохранено 250 вопросов.
Сохранено 260 вопросов.
Сохранено 270 вопросов.
Сохранено 280 вопросов.
Окончательное сохранение завершено.


In [35]:
df_temp = pd.read_parquet('batch_output.parquet')

In [38]:
client = OpenAI()
def complete(query: str, instruction: str):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=1,
        messages=[
            {"role": "system", "content": instruction},
            {
                "role": "user",
                "content": query
            }
        ]
    )
    return completion

def extract_content(completion) -> str:
    try:
        return completion.choices[0].message.content
    except:
        return '==ERROR=='

In [39]:
import os

In [40]:
# Параметры
n = 10  # Количество итераций перед сохранением
output_file = '../data/q1.parquet'

# Список вопросов
new_questions = []

# Загружаем данные, если файл уже существует
if os.path.exists(output_file):
    existing_data = pd.read_parquet(output_file).values.tolist()
    new_questions.extend(existing_data)

# Основной цикл обработки вопросов
for idx, question in enumerate(questions):
    temp = []
    temp.append(question)
    for prompt in prompts:
        temp.append(extract_content(complete(question, prompt)))
    
    new_questions.append(temp)
    
    # Сохраняем каждые n итераций
    if (idx + 1) % n == 0:
        df = pd.DataFrame(new_questions)
        df.to_parquet(output_file)
        print(f"Сохранено {idx + 1} вопросов.")

# Окончательное сохранение, если осталось что-то после последней итерации
df = pd.DataFrame(new_questions)
df.to_parquet(output_file)
print("Окончательное сохранение завершено.")

Сохранено 10 вопросов.
Сохранено 20 вопросов.
Сохранено 30 вопросов.
Сохранено 40 вопросов.
Сохранено 50 вопросов.
Сохранено 60 вопросов.
Сохранено 70 вопросов.
Сохранено 80 вопросов.
Сохранено 90 вопросов.
Сохранено 100 вопросов.
Сохранено 110 вопросов.
Сохранено 120 вопросов.
Сохранено 130 вопросов.
Сохранено 140 вопросов.
Сохранено 150 вопросов.
Сохранено 160 вопросов.
Сохранено 170 вопросов.
Сохранено 180 вопросов.
Сохранено 190 вопросов.
Сохранено 200 вопросов.
Сохранено 210 вопросов.
Сохранено 220 вопросов.
Сохранено 230 вопросов.
Сохранено 240 вопросов.
Сохранено 250 вопросов.
Сохранено 260 вопросов.
Сохранено 270 вопросов.
Сохранено 280 вопросов.
Окончательное сохранение завершено.


In [43]:
tgf = pd.read_parquet('../data/q1.parquet')

In [44]:
tgf

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,Что нельзя публиковать на RUTUBE?,Какие материалы запрещено размещать на платфор...,"Эй, чуваки! Что точно нельзя выкладывать на RU...",Какие материалы не следует выкладывать на RUTU...,Какие категории контента стоит избегать при пу...,"Хей, девчонки! Поделитесь, что точно нельзя за...",Почему на RUTUBE нельзя выкладывать некоторые ...,"А чё нельзя закидывать на этот RUTUBE, брат?",Какие материалы запрещены для размещения на RU...,"Каковы ограничения для контента, который нельз...","Эй, братец, скажи-ка мне, а что ж такого нельз...",Какие материалы строго запрещено залипать на R...,"Что нельзя выкладывать на RUTUBE, чтобы не пол...",Что же запрещено выкладывать на RUTUBE?
1,Почему могут отключить монетизацию из-за автор...,Что может стать причиной отключения монетизаци...,Почему у меня могут отобрать монетизацию на ви...,"Скажите, милочки, что может произойти с моими ...",Как я могу защитить свои видео от отключения м...,Почему может быть блокирована монетизация моих...,Почему видео могут перестать зарабатывать день...,"Эй, братан, скажи на чём стоит по этому вопрос...",Почему монетизацию видео могут отключить из-за...,Почему могут заблокировать монетизацию на виде...,"Скажи, брат, почему ж может в шоу-бизнесе накр...",Почему могут отрубить монетизацию на моём виде...,Почему могут забрать деньги с видео из-за авто...,Почему иногда отключают возможность зарабатыва...
2,Почему могут отключить монетизацию из-за искус...,В чем причина отключения монетизации из-за неп...,"Почему могут отключить монетизацию на видосах,...","Дорогие мои, подскажите, почему бывает так, чт...",Как можно избежать отключения монетизации на в...,"Почему иногда отключают монетизацию, если в ви...","Почему могут прекратить платить за видео, если...","Слышь, а чё за дела, почему могут бабло отжать...",Почему может прекратиться монетизация на видео...,Почему на нашем любимом рок-канале могут отруб...,"Эй, браток, вопрос у меня к тебе такой: отчего...","Почему могут отрубить бабло за видосы, если я ...","Почему могут заблокировать заработок на видео,...","Почему могут отключить заработок на видео, есл..."
3,"Для каких статусов доступна монетизация, и поч...","Какие статусы позволяют включить монетизацию, ...","Эй, в каком режиме можно включить монетизацию ...","Скажите, пожалуйста, для каких категорий людей...",Какие категории аккаунтов могут участвовать в ...,"Эй, девчонки! Интересно, при каких статусах мо...",Почему некоторые видео могут зарабатывать день...,"Слыш, братан, за какие статусы монету можно ру...","Какие статусы позволяют зарабатывать на видео,...",Для каких статусов можно прокачать монетизацию...,"Слушай, брат, расскажи-ка, какие такие статусы...",Какие статусы позволяют заработать на контенте...,"Как мне, многодетному отцу, который старается ...","Привет, дорогие мамочки! У меня возник вопрос:..."
4,Какой контент можно использовать для монетизац...,Какие виды контента можно применять для получе...,"Эй, какой контент можно юзать, чтобы поднять б...",Какой контент можно было бы использовать для т...,Какие виды контента можно создать для эффектив...,"Эй, девчонки! Какие классные идеи контента мож...","Какой материал можно использовать, чтобы на не...","Че, братан, какой контент можно замутить, чтоб...",Какой тип контента можно использовать для зара...,"Какой рок-контент можно крутить, чтобы заработ...","Эй, браток, скажи-ка, какой контент можно заде...","Какой контент можно заюзать для того, чтобы на...","Какой вид контента мне можно использовать, что...","Какой контент можно создавать, чтобы потом его..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,"Видео недавно загрузилось, почему оно до сих п...",Почему недавно загруженное видео еще не отобра...,"Эй, почему моё видео так долго не появляется н...","Сынок, я только что загрузила одно видео, но п...","Я заметил, что видео, которое я загрузил, еще ...","Эй, ребята! Я вот только что заливала новое ви...","Почему видео, которое я загрузил, еще н